Notebook created: 2018-05-23 03:37:56  
Generated from: _build_jl/jl/stationary_densities.rst  

In [ ]:
#=

@author : Spencer Lyon <spencer.lyon@nyu.edu>
          Victoria Gregory <victoria.gregory@nyu.edu>

=#
using QuantEcon
using Distributions
using Plots
using LaTeXStrings

s = 0.2
δ = 0.1
a_σ = 0.4                    # A = exp(B) where B ~ N(0, a_σ)
α = 0.4                      # We set f(k) = k**α
ψ_0 = Beta(5.0, 5.0)         # Initial distribution
ϕ = LogNormal(0.0, a_σ)


function p(x, y)
    #=
    Stochastic kernel for the growth model with Cobb-Douglas production.
    Both x and y must be strictly positive.
    =#
    d = s * x.^α

    # scipy silently evaluates the pdf of the lognormal dist at a negative
    # value as zero. It should be undefined and Julia recognizes this.
    pdf_arg = clamp.((y .- (1-δ) .* x) ./ d, eps(), Inf)
    return pdf.(ϕ, pdf_arg) ./ d
end


n = 10000  # Number of observations at each date t
T = 30     # Compute density of k_t at 1,...,T+1

# Generate matrix s.t. t-th column is n observations of k_t
k = Array{Float64}(n, T)
A = rand!(ϕ, Array{Float64}(n, T))

# Draw first column from initial distribution
k[:, 1] = rand(ψ_0, n) ./ 2  # divide by 2 to match scale=0.5 in py version
for t=1:T-1
    k[:, t+1] = s*A[:, t] .* k[:, t].^α + (1-δ) .* k[:, t]
end

# Generate T instances of LAE using this data, one for each date t
laes = [LAE(p, k[:, t]) for t=T:-1:1]

# Plot
ygrid = linspace(0.01, 4.0, 200)
laes_plot = []
colors = []
for i = 1:T
    ψ = laes[i]
    push!(laes_plot, lae_est(ψ , ygrid))
    push!(colors,  RGBA(0, 0, 0, 1 - (i - 1)/T))
end
plot(ygrid, laes_plot, color=reshape(colors, 1, length(colors)), lw=2, xlabel="capital", legend=:none)
t = LaTeXString("Density of \$k_1\$ (lighter) to \$k_T\$ (darker) for \$T=$T\$")
plot!(title=t)

```none
ψ_0 = Beta(5.0, 5.0)  # Initial distribution
n = 1000
# .... more setup

for i=1:4
    # .... some code
    rand_draws = (rand(ψ_0, n) .+ 2.5i) ./ 2
```


In [ ]:
using StatPlots     # needed for box plot support

n = 500
x = randn(n)  # N(0, 1)
x = exp.(x)   # Map x to lognormal
y = randn(n) + 2.0  # N(2, 1)
z = randn(n) + 4.0  # N(4, 1)
data = vcat(x, y, z)
l = [LaTeXString("\$X\$") LaTeXString("\$Y\$")  LaTeXString("\$Z\$") ]
xlabels = reshape(repmat(l, n), n*3, 1)

boxplot(xlabels, data, label="", ylims=(-2, 14))

```julia
initial_conditions = linspace(8, 0, J)
```


In [ ]:
using KernelDensity

In [ ]:
ϕ = Normal(0.0, 1.0)
n = 500
θ = 0.8
d = sqrt(1.0 - θ^2)
δ = θ / d
srand(41)  # reproducible results

# true density of TAR model
ψ_star(y) = 2 .* pdf.(ϕ, y) .* cdf.(ϕ, δ * y)

# Stochastic kernel for the TAR model.
p_TAR(x, y) = pdf.(ϕ, (y .- θ .* abs.(x)) ./ d) ./ d

Z = rand(ϕ, n)
X = zeros(n)
for t=1:n-1
    X[t+1] = θ * abs(X[t]) + d * Z[t]
end

ψ_est(a) = lae_est(LAE(p_TAR, X), a)
k_est = kde(X)

ys = linspace(-3, 3, 200)
plot(ys, ψ_star(ys), color=:blue, lw=2, alpha=0.6, label="true")
plot!(ys, ψ_est(ys), color=:green, lw=2, alpha=0.6, label="look ahead estimate")
plot!(k_est.x, k_est.density, color=:black, lw=2, alpha=0.6, label="kernel based estimate")

In [ ]:
s = 0.2
δ = 0.1
a_σ = 0.4  # A = exp(B) where B ~ N(0, a_σ)
α = 0.4    # We set f(k) = k**α
ψ_0 = Beta(5.0, 5.0)  # Initial distribution
ϕ = LogNormal(0.0, a_σ)
srand(42)  # reproducible results


function p_growth(x, y)
    #=
    Stochastic kernel for the growth model with Cobb-Douglas production.
    Both x and y must be strictly positive.
    =#
        d = s * x.^α

    # scipy silently evaluates the pdf of the lognormal dist at a negative
    # value as zero. It should be undefined and Julia recognizes this.
    pdf_arg = clamp.((y .- (1-δ) .* x) ./ d, eps(), Inf)
    return pdf.(ϕ, pdf_arg) ./ d
end


n = 1000  # Number of observations at each date t
T = 40    # Compute density of k_t at 1,...,T+1

xmax = 6.5
ygrid = linspace(0.01, xmax, 150)
laes_plot = zeros(length(ygrid), 4*T)
colors = []
for i=1:4
    k = Array{Float64}(n, T)
    A = rand!(ϕ, Array{Float64}(n, T))

    # Draw first column from initial distribution
    # match scale=0.5 and loc=2*i in julia version
    k[:, 1] = (rand(ψ_0, n) .+ 2.5i) ./ 2
    for t=1:T-1
        k[:, t+1] = s*A[:, t] .* k[:, t].^α + (1-δ) .* k[:, t]
    end

    # Generate T instances of LAE using this data, one for each date t
    laes = [LAE(p_growth, k[:, t]) for t=T:-1:1]
    ind = i
    for j = 1:T
        ψ = laes[j]
        laes_plot[:, ind] = lae_est(ψ, ygrid)
        ind = ind + 4
        push!(colors,  RGBA(0, 0, 0, 1 - (j - 1)/T))
    end
end

#colors = reshape(reshape(colors, T, 4)', 4*T, 1)
colors=reshape(colors, 1,length(colors))
plot(ygrid, laes_plot, layout=(2,2), color=colors,
     legend=:none, xlabel="capital", xlims=(0, xmax))

In [ ]:
n = 20
k = 5000
J = 6
srand(43)  # reproducible results

θ = 0.9
d = sqrt(1 - θ^2)
δ = θ / d

initial_conditions = linspace(8, 0, J)

Z = randn(k, n, J)
titles = []
data = []
x_labels = []
for j=1:J
    title = "time series from t = $(initial_conditions[j])"
    push!(titles, title)

    X = Array{Float64}(k, n)
    X[:, 1] = initial_conditions[j]
    labels = []
    labels = vcat(labels, ones(k, 1))
    for t=2:n
        X[:, t] = θ .* abs.(X[:, t-1]) .+ d .* Z[:, t, j]
        labels = vcat(labels, t*ones(k, 1))
    end
    X = reshape(X, n*k, 1)
    push!(data, X)
    push!(x_labels, labels)
end

In [ ]:
boxplot(x_labels, data, layout=(J,1), title=reshape(titles,1,length(titles)), ylims=(-4, 8),
legend=:none, yticks=-4:2:8, xticks=1:20)
plot!(size=(800, 2000))